In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


In [ ]:
%%capture
!pip3 install flair

## **NER**

In [ ]:
!git clone https://github.com/nasrin-taghizadeh/NSURL-Persian-NER
data_folder = "/content/gdrive/MyDrive/project/data/ner/"

Cloning into 'NSURL-Persian-NER'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 33 (delta 7), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (33/33), done.


In [ ]:
%%capture
!unrar x /content/NSURL-Persian-NER/Farsi-NER-TrainingData.rar 
!rm -rf /content/test-label
!rm -rf /content/test
!unzip /content/NSURL-Persian-NER/Farsi-NER-TestData-tokens.zip -d test
!unzip /content/NSURL-Persian-NER/Farsi-NER-TestData-gold.zip -d test-label

In [ ]:
lines = []
import os
for filename in os.listdir("/content/NER/300K"):
   with open(os.path.join("/content/NER/300K", filename), 'r') as f:
       lines.extend(f.readlines())
for filename in os.listdir("/content/NER/600K"):
   with open(os.path.join("/content/NER/600K", filename), 'r') as f:
       lines.extend(f.readlines())
print(len(lines))

912426


In [ ]:
all_test = []
for filename in os.listdir("/content/test"):
    with open(os.path.join("/content/test", filename), 'r') as f1:
        with open(os.path.join("/content/test-label", filename.replace("txt","gold")), 'r') as f2:
            lines1 = f1.readlines()
            lines2 = f2.readlines()
            for i in range(len(lines1)):
                all_test.append(lines1[i].replace("\n","") + "\t" + lines2[i].replace("\n","") + "\n")
print(len(all_test))
print(all_test[1000:1100])

148676
['،\tO\n', 'خنداندن\tO\n', 'مردم\tO\n', 'ایران\tB-LOC\n', 'به\tO\n', 'همین\tO\n', 'دلیل\tO\n', 'خیلی\tO\n', 'سخت\tO\n', 'شده\tO\n', 'است\tO\n', 'و\tO\n', 'شاید\tO\n', 'یکی\tO\n', 'از\tO\n', 'ریشه های\tO\n', 'کمتر\tO\n', 'توجه\tO\n', 'شده\tO\n', 'در\tO\n', 'کاهش\tO\n', 'سطح\tO\n', 'شاد\tO\n', 'بودن\tO\n', 'عمومی\tO\n', 'همین\tO\n', 'مسیله\tO\n', 'فقر\tO\n', 'محتوای\tO\n', 'باب\tO\n', 'میل\tO\n', 'ذهن\tO\n', 'مخاطب\tO\n', 'باشد\tO\n', '.\tO\n', '\t\n', 'صابر\tB-PER\n', 'ترحمی\tI-PER\n', 'پژوهشگر\tO\n', 'مسایل\tO\n', 'اجتماعی\tO\n', 'می گوید\tO\n', ':\tO\n', 'در\tB-DAT\n', '2\tI-DAT\n', 'دهه\tI-DAT\n', 'اخیر\tI-DAT\n', 'تحقیقات\tO\n', 'مستندی\tO\n', 'از\tO\n', 'بررسی های\tO\n', 'دانشگاهی\tO\n', 'در\tO\n', 'خصوص\tO\n', 'تلاش\tO\n', 'ضد\tO\n', 'فرهنگی\tO\n', 'غرب\tO\n', 'برای\tO\n', 'توهین\tO\n', 'به\tO\n', 'اقوام\tO\n', 'مختلف\tO\n', 'ایرانی\tO\n', 'از\tO\n', 'طریق\tO\n', 'به\tO\n', 'سخره\tO\n', 'گرفتن\tO\n', 'برخی\tO\n', 'حرکات\tO\n', 'و\tO\n', 'رفتارها\tO\n', '،\tO\n', 'حرف ها\tO\

In [ ]:
number_of_sen = 0
# number of sentences is 27130
SENTENCE_NUMBERS = 27130
train = []
valid = []
label_keys = {}
i = 0
for line in lines:
    if len(line.split(" ")) > 1 :
        line = line.replace(" ", "")
    if len(line.split("\t")) > 1 and "ر" in line.split("\t")[1]:
        # print(line)
        continue
    if line == "\n":
        number_of_sen += 1
    elif len(line.split("\t")) <= 1:
        continue
    else:
        part1 = line.split('\t')[0]
        part2 = line.split('\t')[1].replace('_' , '-')
        if len(part2)>6:
            part2 = part2[0:5] + '\n'
        if part2 == "OB-OR\n":
            part2 = "B-ORG\n"
        line = part1 + "\t" + part2
        if not line.split('\t')[1] in label_keys.keys():
            label_keys[line.split('\t')[1]] = 0
        label_keys[line.split('\t')[1]] += 1
    if number_of_sen < (SENTENCE_NUMBERS * 85) / 100:
        train.append(line)
    elif number_of_sen >= (SENTENCE_NUMBERS * 85) / 100:
        valid.append(line)
    i+=1
print(number_of_sen)
print(label_keys)

27130
{'O\n': 788592, 'B-PER\n': 12557, 'I-PER\n': 8564, 'B-ORG\n': 14285, 'I-ORG\n': 20489, 'B-TIM\n': 572, 'I-TIM\n': 1214, 'B-LOC\n': 15412, 'B-DAT\n': 4474, 'I-LOC\n': 5690, 'B-PCT\n': 997, 'I-PCT\n': 1389, 'I-DAT\n': 5939, 'B-MON\n': 1295, 'I-MON\n': 3431}


In [ ]:
test = []
label_keys = {}
for line in all_test:
    if len(line.split(" ")) > 1 :
        line = line.replace(" ", "")
    if len(line.split("\t")) > 1 and "ر" in line.split("\t")[1]:
        print(line)
        continue
    if line == "\n":
        number_of_sen += 1
    elif len(line.split("\t")) <= 1:
        print(line)
        continue
    else:
        part1 = line.split('\t')[0]
        part2 = line.split('\t')[1].replace('_' , '-')
        if len(part2)>6:
            part2 = part2[0:5] + '\n'
        if part2 == "OB-OR\n":
            part2 = "B-ORG\n"
        line = part1 + "\t" + part2
        if not line.split('\t')[1] in label_keys.keys():
            label_keys[line.split('\t')[1]] = 0
        label_keys[line.split('\t')[1]] += 1
    test.append(line)
number_of_sen
print(test[1:10])
print(label_keys)

['گزارش\tO\n', 'ایسنا\tB-ORG\n', '،\tO\n', 'در\tO\n', 'دوره\tO\n', 'گارانتی\tO\n', '(\tO\n', 'تضمین\tO\n', ')\tO\n']
{'O\n': 121076, 'B-ORG\n': 3165, '\n': 4150, 'B-LOC\n': 4083, 'I-ORG\n': 4919, 'B-PER\n': 2741, 'I-PER\n': 2138, 'B-TIM\n': 166, 'I-TIM\n': 234, 'I-LOC\n': 1747, 'B-DAT\n': 1150, 'I-DAT\n': 1612, 'B-PCT\n': 156, 'I-PCT\n': 176, 'B-MON\n': 357, 'I-MON\n': 806}


In [ ]:
with open(data_folder + 'train.txt', 'w', encoding="utf-8") as f:
    for item in train:
        f.write(item)
with open(data_folder + 'valid.txt', 'w', encoding="utf-8") as f:
    for item in valid:
        f.write(item)
with open(data_folder + 'test.txt', 'w', encoding="utf-8") as f:
    for item in test:
        f.write(item)

In [ ]:
print(len(train) , " ",len(test) , " ",len(valid) , " ")

769242   148676   142788  


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'ner'}

corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='valid.txt').downsample(1)

2022-03-27 02:34:11,927 Reading data from /content/gdrive/MyDrive/project/data/ner
2022-03-27 02:34:11,929 Train: /content/gdrive/MyDrive/project/data/ner/train.txt
2022-03-27 02:34:11,934 Dev: /content/gdrive/MyDrive/project/data/ner/valid.txt
2022-03-27 02:34:11,937 Test: /content/gdrive/MyDrive/project/data/ner/test.txt


In [ ]:
print(corpus.test[301])

Sentence: "او در سال 1362 و با هدف پایان دادن جنگ ایران و عراق به تهران سفر کرد و با آیت الله خامنهای رییسجمهور و علی اکبر ولایتی وزیر خارجه وقت دیدار و گفتگو کرد ."   [− Tokens: 35  − Token-Labels: "او در سال <B-DAT> 1362 <I-DAT> و با هدف پایان دادن جنگ ایران <B-LOC> و عراق <B-LOC> به تهران <B-LOC> سفر کرد و با آیت الله خامنهای <B-PER> رییسجمهور و علی <B-PER> اکبر <I-PER> ولایتی <I-PER> وزیر خارجه <B-ORG> وقت دیدار و گفتگو کرد ."]


In [ ]:
print(len(corpus.train))
print(len(corpus.test))
print(len(corpus.dev))

23060
4150
4070


In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-03-21 22:01:37,831 Computing label dictionary. Progress:


100%|██████████| 23060/23060 [00:02<00:00, 7908.09it/s]

2022-03-21 22:01:40,832 Corpus contains the labels: ner (#746175)


2022-03-21 22:01:40,838 Created (for label 'ner') Dictionary with 16 tags: <unk>, O, B-ORG, I-ORG, B-LOC, I-LOC, B-PER, B-DAT, I-PER, B-TIM, I-DAT, I-TIM, B-MON, I-MON, B-PCT, I-PCT
Dictionary with 16 tags: <unk>, O, B-ORG, I-ORG, B-LOC, I-LOC, B-PER, B-DAT, I-PER, B-TIM, I-DAT, I-TIM, B-MON, I-MON, B-PCT, I-PCT


In [ ]:
label_dict.remove_item('در')
print(label_dict.get_items())

['<unk>', 'O', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PER', 'B-DAT', 'I-PER', 'B-TIM', 'I-DAT', 'I-TIM', 'B-MON', 'I-MON', 'B-PCT', 'I-PCT']


In [ ]:
print(corpus.train[0].to_tagged_string('ner'))

تظاهرات فلسطینیان در اعتراض به انتقال سفارت <B-ORG> آمریکا <I-ORG> به قدس <B-LOC> اشغالی <I-LOC>
